# Debugging

Cluster **data-import** started at 9:00

In [1]:
import coiled
import dask.dataframe as dd
from dask.distributed import Client
import dask

In [ ]:
# coiled.create_software_environment(
#     name='create-coiled-datasets-m1-test',
#     conda='/Users/rpelgrim/Documents/git/coiled-resources/create-coiled-datasets/environment-m1-test.yml'
# )

In [2]:
cluster = coiled.Cluster(
    name="data-import-2",
    software='rrpelgrim/create-coiled-datasets-m1-test',
    n_workers=10,
    worker_cpu=4,
    worker_memory='16Gb',
    scheduler_cpu=2,
    shutdown_on_close=False,
    scheduler_options={'idle_timeout':'1hour'},
)

Output()

Found software environment build
Created FW rules: coiled-dask-rrpelgr71-36493-firewall
Created scheduler VM: coiled-dask-rrpelgr71-36493-scheduler (ip: ['3.85.234.70'])


In [3]:
client = Client(cluster)
client

<Client: 'tls://10.4.0.184:8786' processes=7 threads=28, memory=98.00 GiB>

## This works

In [5]:
from pandas import DataFrame, to_datetime
from dask.dataframe import from_pandas

df = DataFrame({
     "index": ["1", "2", "3", "2", "3", "1", "4"],
     "date":  to_datetime(["2017-01-01", "2017-01-01", "2017-01-01", "2017-01-02", "2017-01-02", 
                                            "2017-01-06", "2017-01-09"]),
     "amount": [100, 200, 300, 400, 500, 600, 700]
    }
)

df = from_pandas(df, npartitions=2).set_index("index")

In [9]:
df.to_csv(
    "s3://coiled-datasets/dea-opioid/tests/mre-dummy-test-170821.csv",
)

['coiled-datasets/dea-opioid/tests/mre-dummy-test-170821.csv/0.part',
 'coiled-datasets/dea-opioid/tests/mre-dummy-test-170821.csv/1.part']

In [10]:
df.to_parquet(
    "s3://coiled-datasets/dea-opioid/tests/mre-dummy-test.-170821parquet",
    engine='pyarrow',
    schema='infer'
)

## This Fails

In [4]:
data = dask.datasets.timeseries("1950")
#data = data.repartition(partition_size='100MB')
data.head()

,id,name,x,y
timestamp,,,,
1950-01-01 00:00:00,1004,Zelda,-0.539252,0.670639
1950-01-01 00:00:01,1051,Ray,-0.952283,0.958458
1950-01-01 00:00:02,1020,Kevin,-0.062935,0.436572
1950-01-01 00:00:03,975,Michael,0.059545,0.423347
1950-01-01 00:00:04,1029,Charlie,-0.910200,-0.813256


In [7]:
data.to_parquet(
    "s3://coiled-datasets/dea-opioid/tests/dummy-timeseries-170821-2.parquet",
    engine='pyarrow',
    schema='infer'
)

KilledWorker: ("('make-timeseries-71ccf531b2d935357865aa88c20041ce', 12925)", <WorkerState 'tls://10.4.1.99:33489', name: coiled-rrpelgr71-36483-worker-9958ba21ed, memory: 0, processing: 194>)

In [ ]:
data.to_csv(
    "s3://coiled-datasets/dea-opioid/tests/dummy-170821.csv",
)

In [ ]:
data = data.repartition(partition_size='100MB')
data

## Workaround

In [7]:
def submit_jobs():
    from distributed import get_client
    
    with get_client() as client:
        data = dask.datasets.timeseries(
            start="1950",
            end="1951",
        )
        data.to_parquet(
            "s3://coiled-datasets/dea-opioid/tests/dummy-timeseries-170821-2.parquet",
            engine='fastparquet',
        )
        data.compute()

In [8]:
client.submit(submit_jobs).result()

In [8]:
df = dd.read_parquet(
    "s3://coiled-datasets/nyc-taxi/yellow_tripdata_2019.parquet/part.0.parquet", 
    blocksize='16MB',
    dtype={
        "VendorID": "UInt8",
        "passenger_count": "UInt8",
        "RatecodeID": "UInt8",
        "store_and_fwd_flag": "category",
        "PULocationID": "UInt16",
        "DOLocationID": "UInt16",
        "payment_type": "UInt8",
    },
)

df.to_parquet(
    "s3://coiled-datasets/dea-opioid/tests/nyc-test.parquet",
    engine='pyarrow'
)

In [9]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,N,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,1,N,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.0,1,N,236,236,1,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.0,1,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,7.55,NaN
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.0,2,N,193,193,2,52.0,0.0,0.5,0.00,0.0,0.3,55.55,NaN


In [7]:
df.to_parquet(
    "s3://coiled-datasets/dea-opioid/tests/nyc-test.parquet",
    engine='pyarrow'
)